In [97]:
#importing all the required libraries
import matplotlib.pyplot as plt
import cv2
import numpy as np
import scipy as sio
from scipy import io
import itertools
from numpy import linalg as LA
from scipy.optimize import least_squares

In [98]:
#importing the given .mat files
Intmat_contents=sio.io.loadmat("D:\IIT MADRAS-COLLEGE WORK\Sem 1(July- Nov '22)\EE5178 Modern Computer Vision\Assignment5_SFM\IntrinsicsMCV.mat");
Ptsmat_contents=sio.io.loadmat("D:\IIT MADRAS-COLLEGE WORK\Sem 1(July- Nov '22)\EE5178 Modern Computer Vision\Assignment5_SFM\MatchedPoints.mat");

In [99]:
#printing intrinsic matrix given
test_struct = Intmat_contents['IntrinsicMatrix']
print(test_struct)

[[720.6605049    0.         420.22416865]
 [  0.         720.05791688 316.64872007]
 [  0.           0.           1.        ]]


In [100]:
global K, Pts_Struct

In [101]:
#printing the matched points given
Pts_struct = Ptsmat_contents['matchedPoints']
print(Pts_struct)

[[[ 19.70926476  18.91908836  18.27763176 ...  14.43491554  13.26954651
    13.25377846]
  [109.77523041 102.79807281  99.92861938 ... 118.55937958 106.96743774
   104.78275299]]

 [[ 25.29519272  24.52516174  23.88777161 ...  20.64873123  19.46090698
    19.45031357]
  [114.76313782 107.79125214 104.86302185 ... 123.74604797 112.09181976
   109.87740326]]

 [[ 32.80758286  32.11218262  31.48503876 ...  28.19917488  27.04652405
    26.97327423]
  [110.10574341 103.08643341 100.11097717 ... 119.30238342 107.39344788
   105.22314453]]

 ...

 [[807.72125244 808.887146   809.02966309 ... 799.1260376  799.68048096
   799.49023438]
  [102.71156311  98.81201172  96.79473877 ... 117.65009308 115.01673126
   115.19567108]]

 [[810.13916016 811.28875732 811.38604736 ... 801.80224609 802.43243408
   802.27398682]
  [108.5565567  104.64221954 102.62509918 ... 123.50378418 120.83622742
   121.01147461]]

 [[810.2598877  811.1751709  811.20446777 ... 800.92333984 800.72607422
   800.5032959 ]
  [14

In [102]:
#checking the number of points given 
print(Pts_struct.shape)

(807, 2, 25)


In [103]:
#acessing elements from the intrinsic matrix given
Fx = test_struct[0][0]
Fy = test_struct[1][1]
Cx = test_struct[0][2]
Cy = test_struct[1][2]
print('Fx= ',Fx,'Fy= ', Fy, 'Cx= ',Cx,'Cy= ', Cy)

Fx=  720.6605048966444 Fy=  720.0579168844923 Cx=  420.22416864692804 Cy=  316.64872007180253


In [104]:
#defining a function to convert the given keypoints to normalized camera co-ordinates

def normalizeKeyPoint(matchedPoints,fx,fy,cx,cy):
    N_pts,xy,N_camera=matchedPoints.shape
    result=np.zeros((N_pts,xy,N_camera))
  
    for pt in range(0,N_pts):
        for camera_pts in  range(0,N_camera):
            result[pt,0,camera_pts]=(matchedPoints[pt,0,camera_pts]-cx)/fx
            result[pt,1,camera_pts]=(matchedPoints[pt,1,camera_pts]-cy)/fy
     
    return result

In [105]:
#calculating the normalized points using function defined
Norm_Points = normalizeKeyPoint(Pts_struct,Fx,Fy,Cx,Cy)

In [106]:
K = test_struct

In [107]:
#caluclating the inverse of the intrinsic matrix for multiplication
Kinv= np.linalg.inv(K)
print(Kinv)

[[ 0.00138762  0.         -0.58310975]
 [ 0.          0.00138878 -0.43975452]
 [ 0.          0.          1.        ]]


In [108]:
#function to convert 2D points to 3D world co-ordinates 
def Point_Conversion(matchedPoints, K):
    N_pts,xy,N_camera=matchedPoints.shape
    Pts_3D=np.zeros((N_pts,xy+1,N_camera))
    for pt in range(0,N_pts):
        for camera_pts in  range(0,N_camera):
            Pts_3D[pt,0,camera_pts]=((matchedPoints[pt,0,camera_pts])*(K[0][0])) + ((matchedPoints[pt,1,camera_pts])*(K[0][1]) + (K[0][2]))
            Pts_3D[pt,1,camera_pts]=((matchedPoints[pt,0,camera_pts])*(K[1][0])) + ((matchedPoints[pt,1,camera_pts])*(K[1][1]) + (K[1][2]))
            Pts_3D[pt,2,camera_pts]=((matchedPoints[pt,0,camera_pts])*(K[2][0])) + ((matchedPoints[pt,1,camera_pts])*(K[2][1]) + (K[2][2]))    
            
    return Pts_3D

In [109]:
Pts3D_Ref = Point_Conversion(Norm_Points, Kinv)

In [110]:
print(Pts3D_Ref)

[[[-0.58388094 -0.58388246 -0.58388369 ... -0.58389109 -0.58389333
   -0.58389337]
  [-0.44015351 -0.44016697 -0.4401725  ... -0.44013657 -0.44015893
   -0.44016314]
  [ 1.          1.          1.         ...  1.          1.
    1.        ]]

 [[-0.58387018 -0.58387166 -0.58387289 ... -0.58387913 -0.58388141
   -0.58388143]
  [-0.44014389 -0.44015734 -0.44016299 ... -0.44012657 -0.44014904
   -0.44015332]
  [ 1.          1.          1.         ...  1.          1.
    1.        ]]

 [[-0.58385571 -0.58385705 -0.58385826 ... -0.58386459 -0.58386681
   -0.58386695]
  [-0.44015287 -0.44016641 -0.44017215 ... -0.44013514 -0.44015811
   -0.44016229]
  [ 1.          1.          1.         ...  1.          1.
    1.        ]]

 ...

 [[-0.58236364 -0.58236139 -0.58236112 ... -0.58238019 -0.58237912
   -0.58237948]
  [-0.44016714 -0.44017466 -0.44017855 ... -0.44013832 -0.4401434
   -0.44014306]
  [ 1.          1.          1.         ...  1.          1.
    1.        ]]

 [[-0.58235898 -0.58235

In [111]:
#function to find the transformation between two frames
def get_ProjectionMat(pts1, pts2, K):
    
    F= cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)  
    Kt = np.transpose(K)
    var = np.dot(F[0], Kt)
    E = np.dot(K, var)
    Trans_Mat = cv2.recoverPose(E, pts1, pts2)
    R = Trans_Mat[1]
    t = Trans_Mat[2]
    Ext = np.concatenate((R,t), axis=1)
    P = np.dot(K, Ext)
    return P      

In [112]:
for p in range(24):
    P = get_ProjectionMat(Norm_Points[:,: , 0], Norm_Points[:, :, (p+1)], K)
    Q = np.dot(Pts3D_Ref[:, :, p],P)
    print('P',p,'=',P,'\n')
    print('Q',p,'=',Q,'\n')

P 0 = [[ 7.20661970e+02 -1.50555025e+00  4.20218958e+02 -4.20391537e+02]
 [ 1.53812178e+00  7.20080026e+02  3.16594702e+02 -3.18694134e+02]
 [ 7.38826956e-06  7.50020573e-05  9.99999997e-01 -9.99995933e-01]] 

Q 0 = [[-421.45778752 -316.06661573 -383.70810849  384.73295033]
 [-421.45002159 -316.05970461 -383.70054309  384.72536288]
 [-421.43961107 -316.06619467 -383.69730853  384.72214469]
 ...
 [-420.36434826 -316.07871026 -383.07482357  384.09943216]
 [-420.36097579 -316.07059962 -383.06929814  384.09388226]
 [-420.36069013 -316.01529242 -383.04488361  384.06930643]] 

P 1 = [[ 7.20681993e+02 -6.67337491e-01  4.20186787e+02 -4.19371279e+02]
 [ 8.61475704e-01  7.20191706e+02  3.16343137e+02 -3.11017853e+02]
 [ 5.23166485e-05  4.24285939e-04  9.99999909e-01 -9.99968851e-01]] 

Q 1 = [[-421.17271304 -316.61452956 -383.58349296  380.7633493 ]
 [-421.16492544 -316.60760106 -383.57591081  380.75582724]
 [-421.15440507 -316.61414597 -383.57264301  380.75252303]
 ...
 [-420.07651472 -316.621

In [113]:
from scipy.optimize import least_squares


In [114]:
x0 = np.array([1,1,1,0,0,0,1])


In [115]:
np.array([1,2,3])/2

array([0.5, 1. , 1.5])

In [116]:
def project3Dto2D(point_3D,k,Rx,Ry,Rz,tx,ty,tz,w):
      point_3D=point_3D/w
      t=np.array([tx,ty,tz])
      R=np.matrix([[1,-Rz,Ry],[Rz,1,-Rx],[-Ry,Rx,1]])
      projectedPoint=np.matmul(R,point_3D)+t
      # print(projectedPoint,projectedPoint.shape)
      homo_3D=np.matmul(k,np.transpose(projectedPoint))
      # print(homo_3D,homo_3D.shape)
      return np.array([homo_3D[0,0]/homo_3D[2,0],homo_3D[1,0]/homo_3D[2,0]])

In [117]:
def ReprojectionError(input_array):
    # Rx,Ry,Rz,tx,ty,tz,w=input_array
    total_error = 0
    N_pts,xy,N_camera=Pts_struct.shape
    for pt in range(0,N_pts):
      error_point=0
      for camera_pts in  range(0,N_camera):
        # project3Dto2D(point_cloud[0,:,0],IntrinsicMatrix,1,1,1,0,0,0)
        Rx,Ry,Rz,tx,ty,tz,w=input_array[((7*(camera_pts)),(7 + (7*(camera_pts))))]
        point_3d=np.concatenate((Pts_struct[pt,:,camera_pts],np.array([1])),axis=0)
        error_point=project3Dto2D(point_3d,K,Rx,Ry,Rz,tx,ty,tz,w)-Pts_struct[pt,:,camera_pts]
        total_error=total_error+ (LA.norm(error_point)*LA.norm(error_point))

    return total_error

In [118]:
np.concatenate((np.array([1,2]),np.array([3])),axis=0)

array([1, 2, 3])

In [119]:
from scipy.optimize import least_squares


In [120]:
x0_rosenbrock = np.array([1,1,1,0,0,0,1])
res_1 = least_squares(ReprojectionError, x0_rosenbrock)

KeyboardInterrupt: 